### Experiment Tracking with W&B

- config: store hp and metadata for each run
- wandb.init
- wandb.watch: log model gradients and params over time (helps detect bugs e.g. weird grad behaviour)
- wandb.log: log stuff we care about
- wandb.save: save online

use with block in context manager syntax

In [ ]:
import wandb
wandb.login()

In [ ]:
config = dict(
    epochs = 50,
    val_ratio = 0,
    test_ratio = 0.2
)

In [ ]:
def make(base_path, val_ratio, test_ratio, encode_data_name, decode_data_name, latent_dim):
    # TODO: make edges to device here on when called on
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # dataset to encode
    encode_dataset = ReactionDataset(base_path, geo_file = encode_data_name, dataset_type= 'individual')
    encode_data = encode_dataset.data
    encode_data.train_mask = encode_data.val_mask = encode_data.test_mask = encode_data.y = None
    encode_data = train_test_split_edges(data = encode_data, val_ratio = val_ratio, test_ratio = test_ratio)
    encode_x = encode_data.x.to(device)
    encode_train_pos_edge_index = encode_data.train_pos_edge_index.to(device)

    # dataset to decode
    decode_dataset = ReactionDataset(base_path, geo_file = decode_data_name, dataset_type= 'individual')
    decode_data = decode_dataset.data
    decode_data.train_mask = decode_data.val_mask = decode_data.test_mask = decode_data.y = None
    decode_data = train_test_split_edges(data = decode_data, val_ratio = val_ratio, test_ratio = test_ratio)
    decode_x = decode_data.x.to(device)
    decode_train_pos_edge_index = decode_data.train_pos_edge_index.to(device)

    # model creation
    gae = GAE(MolEncoder(encode_data.num_node_features, latent_dim))
    opt = torch.optim.Adam(gae.parameters(), lr = 0.01)

    return gae, opt, encode_data, decode_data

In [ ]:
def model_pipeline(hps):

    # start wandb
    with wandb.init(project="test", config=hps):
        
        # access hps through wandb.config so logging matches execution
        config = wandb.config

        # model data
        
        val_ratio = 0
        test_ratio = 0.2
        
        # make model, data, opt problem
        ts_r_gae, ts_r_opt, r_data, ts_data = make(r'data/', 0, 0.2, 'train_r', 'train_ts', 2)

### Testing GAEs

In [1]:
# data processing
from ts_vae.data_processors.new_pyg_processor import ReactionDataset

# node AE
from ts_vae.simple_gaes.node_gae import Node_AE
from ts_vae.simple_gaes.node_gae import train_node_ae, test_node_ae

# nodeedge AE
from ts_vae.simple_gaes.nodeedge_gae import NodeEdge_AE
from ts_vae.simple_gaes.nodeedge_gae import train_ne_ae

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F

# torch geometric
from torch_geometric.data import DataLoader
# should double check this func works okay
from torch_geometric.utils import to_dense_adj

# other
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

In [2]:
# remove processed files
import os
import glob

files = glob.glob(r'data/processed/*')
for f in files:
    os.remove(f)

## New try

In [2]:
rxns = ReactionDataset(r'data')
reactants = rxns.data.r
transition_states = rxns.data.ts
products = rxns.data.p

# train_loader = DataLoader(rxns[: num_train], batch_size = 2, follow_batch = ['r', 'p'])
# test_loader = DataLoader(rxns[num_train:], batch_size = 2, follow_batch = ['r', 'p'])

num_rxns = len(rxns)
train_ratio = 0.8
num_train = int(np.floor(train_ratio * num_rxns))

batch_size = 2

# need to be able to recover original reactants after encoding
# note: no padding, since PyG automatically factors this in
train_loaders = {'r':  DataLoader(reactants[: num_train], batch_size), 
                 'ts': DataLoader(transition_states[: num_train], batch_size), 
                 'p':  DataLoader(products[: num_train], batch_size)}

test_loaders =  {'r':  DataLoader(reactants[num_train: ], batch_size), 
                 'ts': DataLoader(transition_states[num_train: ], batch_size), 
                 'p':  DataLoader(products[num_train: ], batch_size)}

In [3]:
### Node AE

# TODO: same for test_loader?
max_num_nodes = max([r.z.size(0) for r in train_loaders['r'].dataset])
assert([r.x.size(1) for r in train_loaders['r'].dataset] == [train_loaders['r'].dataset[0].x.size(1)] * len(train_loaders['r'].dataset))
num_node_fs = train_loaders['r'].dataset[0].x.size(1)
num_edge_fs = train_loaders['r'].dataset[0].edge_attr.size(1)
h_nf = 5
emb_nf = 2

# in_node_nf + in_edge_nf >= h_nf >= out_nf > emb_nf 
node_ae = Node_AE(in_node_nf = num_node_fs, in_edge_nf = num_edge_fs, h_nf = h_nf, out_nf = h_nf, emb_nf = emb_nf)
node_opt = torch.optim.Adam(node_ae.parameters(), lr = 1e-3)

# train and test, add epochs after
train_loss, train_res = train_node_ae(node_ae, node_opt, train_loaders['r'])
test_loss, test_res = test_node_ae(node_ae, node_opt, test_loaders['r']) 

In [4]:
# NodeEdge AE

max_num_nodes = max([r.z.size(0) for r in train_loaders['r'].dataset])
assert([r.x.size(1) for r in train_loaders['r'].dataset] == [train_loaders['r'].dataset[0].x.size(1)] * len(train_loaders['r'].dataset))
num_node_fs = train_loaders['r'].dataset[0].x.size(1)
num_edge_fs = train_loaders['r'].dataset[0].edge_attr.size(1)
h_nf = 5
emb_nf = 2

ne_ae = NodeEdge_AE(in_node_nf = num_node_fs, in_edge_nf = num_edge_fs, h_nf = h_nf, out_nf = h_nf, emb_nf = emb_nf)
ne_opt = torch.optim.Adam(ne_ae.parameters(), lr = 1e-3)

# train and test
train_loss, train_res = train_ne_ae(ne_ae, ne_opt, train_loaders['r'])
test_loss, test_res = test_ne_ae(ne_ae, test_loaders['r']) 

In [21]:
batch = next(iter(test_loaders['r']))

node_feats, edge_index, edge_attr = batch.x, batch.edge_index, batch.edge_attr

# batch.x.shape, batch.x[row].shape, batch.x[col].shape



# in_node_nf * 2 + in_edge_nf

node_is, node_js = edge_index
# get node feats for each bonded pair of atoms
node_is_fs, node_js_fs = node_feats[node_is], node_feats[node_js]
edge_in = torch.cat([node_is_fs, node_js_fs, edge_attr], dim = 1)
node_is_fs.shape, edge_in.shape, edge_attr.shape

edge_mlp = nn.Sequential(nn.Linear(in_node_nf * 2 + in_edge_nf, h_nf, bias = bias),
                                      act_fn,
                                      nn.Linear(h_nf, out_nf, bias = bias))

NameError: name 'in_node_nf' is not defined

tensor([[0., 0., 0., 1., 0., 8., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 2.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 2.],
        [0., 0., 1., 0., 0., 7., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 2.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 2.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [158]:
# R->R: node ae, train 
#   DONE!
# R->R: node + edge ae, train
# R->R: node + edge + coords, train
# same for R->TS, P->TS, (R,P)->TS

# look at DGL for improvements: k-hop graph func + khop adj util func

Batch(batch=[15], edge_attr=[30, 4], edge_index=[2, 30], idx=[1], pos=[15, 3], ptr=[2], x=[15, 11], z=[15])

In [ ]:
def train_gae(gae, opt, x, train_pos_edge_index):
    gae.train()
    opt.zero_grad()
    print("train x shape: ", x.shape)
    z = gae.encode(x, train_pos_edge_index)
    print("train z shape: ", z.shape)
    loss = gae.recon_loss(z, train_pos_edge_index)
    loss.backward()
    opt.step()
    return float(loss)

def test_gae(gae, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
    gae.eval()
    with torch.no_grad():
        z = gae.encode(x, train_pos_edge_index)
    return gae.test(z, test_pos_edge_index, test_neg_edge_index)

def new_test_gae(gae, x, edge_index):
    # this just does recon loss again
    gae.eval()
    with torch.no_grad():
        print("test x shape: ", x.shape)
        z = gae.encode(x, edge_index)
        print("test z shape: ", z.shape)
    return gae.recon_loss(z, edge_index)

r_ae.reset_parameters()

epochs = 10
for epoch in range(1, epochs + 1):

    # value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim = 1)
    loss_train = train_gae(r_ae, r_opt, r_x, r_data.edge_index)
    print("===== Training complete with loss: {:.4f}, now testing ====".format(loss_train))
    loss_test = new_test_gae(r_ae, test_x, test_data.edge_index)
    if epoch % 1 == 0:
        print('===== Epoch: {:03d}, Loss: {:.4f} ===== \n'.format(epoch, loss_test))